In [ ]:
#default_exp utils

# Basic Utils

> Basic utility functions across sac_stac.

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import os
import boto3

from sedas_pyapi.sedas_api import SeDASAPI

## Sedas

Probably best if we start by logging into sedas to be able to access datasets in their current format.

In [ ]:
#export
def sedas_client():
    "Log into sedas api."
    sedas = SeDASAPI(os.getenv('SEDAS_USERNAME'), os.getenv('SEDAS_PWD'))
    sedas.base_url="https://geobrowsertest.satapps.org/api/"
    return sedas

In [ ]:
sedas_client()

should inc. test for env vars...?

In [ ]:
#export
def get_sedas_collections():
    sedas = sedas_client()
    result_groups = sedas.list_sensor_groups()
    groups = []
    for i in range(0, len(result_groups)):
        groups.append(result_groups[i]['name'])
    return f"Available groups are: {', '.join(groups)}"

In [ ]:
get_sedas_collections()

'Available groups are: Cosmo-SkyMed, SPOT, Pleiades, S1, S2, AIRSAR'

## Cloud-Optimised Formatting

Once we have access to the datasets, we will want to convert raster images of any format into a default COG format. (It helps that COGs are now an official [gdal driver](https://gdal.org/drivers/raster/cog.html).

In [ ]:
#export
def convert2cog(img_path, cog_path, band):
    """
    Convert gdal raster into COG with default settings and LZW
    compression.
    See https://www.cogeo.org/developers-guide.html.
    """
    # translate into new cog file
    kwargs = {
        'format': 'COG',
#         'creationOptions' : ['COMPRESS=LZW'],
        'bandList': [band]
    }
    ds = gdal.Translate(cog_path, img_path, **kwargs)
    ds = None
    print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Conversion complete: {os.path.exists(cog_path)}.")

In [ ]:
#export
def mosaic2cog(imgs, band):
    """
    mosaic imgs into cog.
    assumes [:13] of first img as output basename,
    + usual vrt assumptions.
    """
    out_cog_mosaic = f"{imgs[0][:-13]}_band{band}.TIF"
    tmp_vrt = f"{imgs[0][:-13]}_mosaic.TIF"
    
    # mosaic into vrt
    print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Mosaicing band {band} imgs: {imgs}")
    ds = gdal.BuildVRT(tmp_vrt, imgs, bandList=[band])

    # convert write vrt mosaic to cog
    print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Converting band {band} mosaic {tmp_vrt} to COG {out_cog_mosaic}")
    convert2cog(ds, out_cog_mosaic, 1)
    ds = None

    if os.path.exists(tmp_vrt):
        os.remove(tmp_vrt)

mosaic currently specific to pleiades ([:-13] for slicing name). Needs to be generalised a little more

## Object Storage

In [ ]:
#export
def s3_create_client(s3_bucket):
    """
    Create and set up a connection to S3
    :param s3_bucket:
    :return: the s3 client object.
    """
    access = os.getenv("AWS_ACCESS_KEY_ID")
    secret = os.getenv("AWS_SECRET_ACCESS_KEY")

    session = boto3.Session(
        access,
        secret,
    )
    endpoint_url = os.getenv("AWS_S3_ENDPOINT_URL")

    if endpoint_url is not None:
        s3 = session.resource('s3', endpoint_url=endpoint_url)
    else:
        s3 = session.resource('s3', region_name='eu-west-2')

    bucket = s3.Bucket(s3_bucket)

    if endpoint_url is not None:
        s3_client = boto3.client(
            's3',
            aws_access_key_id=access,
            aws_secret_access_key=secret,
            endpoint_url=endpoint_url
        )
    else:
        s3_client = boto3.client(
            's3',
            aws_access_key_id=access,
            aws_secret_access_key=secret
        )
    return s3_client, bucket

We can use this to connect to any S3-like object storage end point. Assumes env vars.

In [ ]:
#export
gb = 1024 ** 3

In [ ]:
#export
def s3_single_upload(in_path, s3_path, s3_bucket):
    """
    put a file into S3 from the local file system.
    :param in_path: a path to a file on the local file system
    :param s3_path: where in S3 to put the file.
    :return: None
    """
    # prep session & creds
    s3_client, bucket = s3_create_client(s3_bucket)

    # Ensure that multipart uploads only happen if the size of a transfer is larger than
    # S3's size limit for non multipart uploads, which is 5 GB. we copy using multipart 
    # at anything over 4gb
    transfer_config = boto3.s3.transfer.TransferConfig(multipart_threshold=2 * gb,
                                                       max_concurrency=10,
                                                       multipart_chunksize=2 * gb,
                                                       use_threads=True)
    s3_client.upload_file(in_path, bucket.name, s3_path)

In [ ]:
#export
def get_rel_dir_s3_paths(local_dir, s3_dir):
    """
    returns local path, remote path pair list.
    """
    paths = []
    for subdir, dirs, files in os.walk(local_dir):
        for file in files:
            full_path = os.path.join(subdir, file)
            paths.append([ full_path, s3_dir + full_path[len(local_dir):] ])
    return paths

In [ ]:
#export
def s3_upload_dir(in_dir, s3_bucket, s3_dir):

    paths = get_rel_dir_s3_paths(in_dir, s3_dir)

    upload_list = [(in_path, out_path, s3_bucket)
                   for in_path, out_path in paths]

    for i in upload_list:
        s3_single_upload(i[0], i[1], i[2])

In [ ]:
#export
def s3_list_objects_paths(s3_bucket, prefix):
    """List of paths only returned, not full object responses"""
    client, bucket = s3_create_client(s3_bucket)
    
    return [e['Key'] for p in client.get_paginator("list_objects_v2").paginate(Bucket=s3_bucket, Prefix=prefix) for e in p['Contents']]


In [ ]:
#export
def s3_list_objects(s3_bucket, prefix):
    # prep session & creds
    client, bucket = s3_create_client(s3_bucket)
    response = client.list_objects_v2(Bucket=s3_bucket, Prefix=prefix)

    return response

## STAC

## Export

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_utils.ipynb.
Converted cogifying_sample_data.ipynb.
Converted in-memory__local_static_stac.ipynb.
Converted index.ipynb.
Converted remote_static_stac.ipynb.
